In [225]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from datetime import datetime
import ta
import mplfinance as mpf

# import tensorflow.compat.v1.keras.layers.CuDNNLSTM as CuDNNLSTM

In [226]:
import tensorflow as tf

# Set TensorFlow to only allocate memory on the GPU when needed
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPU


In [227]:
# Load the Bitcoin price data
bitcoin_data = pd.read_csv(r'C:\Users\MrBios\Documents\Development\IA-BTC-2023\csv\bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')

# Drop missing values
bitcoin_data.dropna(inplace=True)


In [228]:
# Keep the necessary columns
bitcoin_data = bitcoin_data[['Timestamp', 'Open', 'High', 'Low', 'Close']]

bitcoin_data = bitcoin_data.tail(10000)

In [229]:
# Convert Timestamp to datetime format
bitcoin_data['Timestamp'] = pd.to_datetime(bitcoin_data['Timestamp'], unit='s')

# Compute RSI
bitcoin_data['RSI'] = ta.momentum.rsi(close=bitcoin_data["Close"], fillna=True)

# Compute MACD
bitcoin_data['MACD'] = ta.trend.macd(close=bitcoin_data["Close"], fillna=True)
bitcoin_data['MACD_signal'] = ta.trend.macd_signal(close=bitcoin_data["Close"], fillna=True)
bitcoin_data['MACD_diff'] = ta.trend.macd_diff(close=bitcoin_data["Close"], fillna=True)


In [230]:
# Prepare the data for training
# Prepare the data for training
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(bitcoin_data[['Open', 'High', 'Low', 'Close']])
# scaled_data = scaler.fit_transform(bitcoin_data[['Open', 'High', 'Low', 'Close', 'Volume_(BTC)', 'Volume_(Currency)', 'RSI', 'MACD', 'MACD_signal', 'MACD_diff']])
scaled_data = scaler.fit_transform(bitcoin_data.drop(columns=['Timestamp']))


In [231]:
timeframe = 60
num_features = scaled_data.shape[1]

In [232]:
timeframe = 60
X = []
y = []
for i in range(timeframe, len(scaled_data) - 60):
    X.append(scaled_data[i - timeframe:i, :])
    y.append(scaled_data[i:i + 60, :4])  # Predicting OHLC columns
X = np.array(X)
y = np.array(y)

In [233]:
X.shape, y.shape

((9880, 60, 8), (9880, 60, 4))

In [234]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y[:, -1, :], test_size=0.2, random_state=42)

# Reshape y_train and y_test
y_train = np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1]))
y_test = np.reshape(y_test, (y_test.shape[0], 1, y_test.shape[1]))

In [235]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=4))  # Output 4 units for OHLC

In [236]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [237]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7904, 60, 8), (7904, 1, 4), (1976, 60, 8), (1976, 1, 4))

In [238]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/50
124/124 [==============================] - 4s 18ms/step - loss: 0.0750 - val_loss: 0.0584
Epoch 2/50
124/124 [==============================] - 1s 11ms/step - loss: 0.0635 - val_loss: 0.0577
Epoch 3/50
124/124 [==============================] - 1s 11ms/step - loss: 0.0617 - val_loss: 0.0580
Epoch 4/50
124/124 [==============================] - 1s 11ms/step - loss: 0.0611 - val_loss: 0.0585
Epoch 5/50
124/124 [==============================] - 1s 11ms/step - loss: 0.0604 - val_loss: 0.0577
Epoch 6/50
124/124 [==============================] - 1s 12ms/step - loss: 0.0602 - val_loss: 0.0582
Epoch 7/50
124/124 [==============================] - 1s 12ms/step - loss: 0.0598 - val_loss: 0.0576
Epoch 8/50
104/124 [========================>.....] - ETA: 0s - loss: 0.0597

In [ ]:
# Save the model and weights
version = "V6"
now = datetime.now()
format_date = now.strftime("%Y-%m-%d-%H-%M")
model.save(f'{version}/models/2-{version}-model_{format_date}.h5')
model.save_weights(f'{version}/models/2-{version}-weights_{format_date}.h5')


In [ ]:
# Make predictions
predictions = model.predict(X_test)

In [ ]:
predictions.shape

In [ ]:
# Scale the predictions back to original values
predictions_scaled = scaler.inverse_transform(predictions.reshape(-1, num_features))


In [ ]:
# Reshape the predictions_scaled array back to its original shape
predictions_scaled = predictions_scaled.reshape(predictions.shape)

In [ ]:
# Reshape y_test array to match the shape of predictions_scaled
y_test_reshaped = y_test.reshape(-1, num_features)

In [ ]:
# Scale y_test_reshaped back to original values
y_test_scaled = scaler.inverse_transform(y_test_reshaped)

In [ ]:
# Reshape y_test_scaled back to its original shape
y_test_scaled = y_test_scaled.reshape(y_test.shape)

In [ ]:
predictions.shape, y_test_reshaped.shape

In [ ]:
# Import the necessary libraries
import mplfinance as mpf

# Convert predictions_scaled to a DataFrame
predictions_df = pd.DataFrame(predictions_scaled[-20:], columns=['Open', 'High', 'Low', 'Close'])

# Create a DataFrame with the timestamps for the last 100 data points
timestamps = last_100_actual_df.index[-20:]
timestamps_df = pd.DataFrame(timestamps, columns=['Date'])

# Concatenate the timestamps DataFrame with the predictions DataFrame
predictions_df = pd.concat([timestamps_df, predictions_df], axis=1)

# Convert the index of predictions_df to DatetimeIndex
predictions_df.index = pd.to_datetime(predictions_df.index)

# Plot the predicted OHLC candles
mpf.plot(predictions_df, type='candle', style='yahoo', title='Predicted OHLC Candles', ylabel='Price')

In [ ]:
# # Scale the predictions back to original values
# predictions = scaler.inverse_transform(predictions)
# y_test_reshaped = scaler.inverse_transform(y_test_reshaped)

# # Reshape y_test_reshaped back to its original shape
# y_test_reshaped = np.reshape(y_test_reshaped, (y_test.shape[0], y_test.shape[1], y_test.shape[2]))

In [ ]:
import ccxt
from datetime import datetime, timedelta
timeframe = '1m'
symbol = 'BTC/USDT'
minutes = 120 
since = datetime.now() - timedelta(minutes=minutes)
since = int(since.timestamp() * 1000)
limit = minutes+1
show_more_graph = False
exchange = ccxt.binance()

In [ ]:
last_60_minutes = exchange.fetch_ohlcv('BTC/USDT', timeframe=timeframe, since=since, limit=limit)
last_60_minutes = pd.DataFrame(last_60_minutes, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
last_60_minutes['Timestamp'] = pd.to_datetime(last_60_minutes['Timestamp'], unit='ms')
# drop Volume column
last_60_minutes.drop(columns=['Volume'], inplace=True)
last_60_minutes.set_index('Timestamp', inplace=True)
last_60_minutes.index = last_60_minutes.index + pd.DateOffset(hours=2)
last_60_minutes.tail(1)

In [ ]:
# Scale the data to be values between 0 and 1
last_60_minutes_scaled = scaler.fit_transform(last_60_minutes[['Open', 'High', 'Low', 'Close']])

# Reshape the data into the shape accepted by the LSTM model
last_60_minutes_scaled = np.reshape(last_60_minutes_scaled, (1, last_60_minutes_scaled.shape[0], last_60_minutes_scaled.shape[1]))

# Make predictions using the LSTM model
pred = model.predict(last_60_minutes_scaled)
# Inverse transform the predictions to the original scale
pred = scaler.inverse_transform(pred)
print(pred)

In [ ]:
# plot with matplotlib
import matplotlib.pyplot as plt
plt.plot(pred[0], label='Predicted')
plt.plot(last_60_minutes.iloc[-1], label='Actual')
plt.legend()
plt.show()

In [ ]:
import mplfinance as mpf

window = 60

# Extract the last 100 data points
last_100_actual = last_60_minutes[-window:]
last_100_predicted = pred[-window:]

# Convert the data to a DataFrame with DatetimeIndex
last_100_actual_df = pd.DataFrame(last_100_actual, columns=['Open', 'High', 'Low', 'Close'])
last_100_actual_df.index = last_100_actual.index
last_100_predicted_df = pd.DataFrame(last_100_predicted, columns=['Open', 'High', 'Low', 'Close'])
last_100_predicted_df.index = last_100_actual.index[-len(last_100_predicted):]  # Use the index of last_100_actual
# concatenate the last_100_actual_df and last_100_predicted_df along columns
last_100_df = pd.concat([last_100_actual_df, last_100_predicted_df], axis=1)


# Convert the "Open" column to float type
last_100_actual_df['Open'] = last_100_actual_df['Open'].astype(float)
last_100_predicted_df['Open'] = last_100_predicted_df['Open'].astype(float)

# Plot the actual and predicted OHLC candles with different styles
mpf.plot(last_100_actual_df, type='candle', style='yahoo', title='Actual OHLC Candles', ylabel='Price')
mpf.plot(last_100_predicted_df, type='candle', style='charles', title='Predicted OHLC Candles', ylabel='Price')


In [ ]:
import mplfinance as mpf
# Create a DataFrame for the real data
real_data = bitcoin_data[['Timestamp', 'Open', 'High', 'Low', 'Close']]
real_data.columns = ['Date', 'Open', 'High', 'Low', 'Close']
real_data.set_index('Date', inplace=True)

# Create a DataFrame for the predicted data
predicted_data = pd.DataFrame(predictions, columns=['Open', 'High', 'Low', 'Close'])
predicted_data.index = real_data.index[-len(predicted_data):]  # Set the same index as the real data

# Concatenate the real and predicted data into a single DataFrame
combined_data = pd.concat([real_data[-100:], predicted_data], axis=0)


In [ ]:
# Define the style for the plot
style = mpf.make_mpf_style(base_mpf_style='yahoo', gridstyle=':', y_on_right=True)


In [ ]:
# Plot the data using mplfinance
mpf.plot(combined_data, type='line', style=style, title='Bitcoin Price', ylabel='Price', ylabel_lower='Volume')
